In [1]:
! pip3 install s3fs

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec None
error: uninstall-no-record-file

× Cannot uninstall fsspec None
╰─> The package's contents are unknown: no RECORD file was found for fsspec.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps fsspec==2023.6.0


# import libraries

In [2]:
import boto3
import pandas as pd 

from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# import input data from S3

In [3]:

role = get_execution_role()

bucket = 'mysagemakerprojects'

data_key = 'cancer-prediction/test_data.csv'

data_location = f's3://{bucket}/{data_key}'

cancer_data = pd.read_csv(data_location)

cancer_data.head(10)

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,survived
1,78,female,breast,1,survived
2,42,female,breast,3,survived
3,38,female,breast,4,died
4,46,female,breast,2,survived
5,73,female,breast,2,survived
6,11,male,lung,3,survived
7,60,male,colon,2,died
8,100,female,colon,1,survived
9,42,male,lung,4,died


In [4]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   age           1000 non-null   int64 
 1   gender        1000 non-null   object
 2   cancer_type   1000 non-null   object
 3   cancer_stage  1000 non-null   int64 
 4   outcome       1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [5]:
cancer_data.isna().sum()

age             0
gender          0
cancer_type     0
cancer_stage    0
outcome         0
dtype: int64

In [6]:
print(cancer_data['outcome'].unique())

['survived' 'died']


In [7]:
cancer_data['outcome'] = cancer_data['outcome'].map({'died': 0, 'survived': 1})


In [8]:
cancer_data.head()

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,1
1,78,female,breast,1,1
2,42,female,breast,3,1
3,38,female,breast,4,0
4,46,female,breast,2,1


In [9]:
# separate features and labels
X = cancer_data.drop('outcome', axis=1)
y = cancer_data['outcome']

# split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri

# save training and validation data to S3
X_train.to_csv('train.csv', index=False, header=False)
X_test.to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket).Object('cancer-prediction/train/train.csv').upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('cancer-prediction/test/test.csv').upload_file('test.csv')

# specify the training data location in S3
train_location = f's3://{bucket}/cancer-prediction/train/train.csv'
test_location = f's3://{bucket}/cancer-prediction/test/test.csv'


In [11]:

train_df = pd.read_csv(train_location, header=None)
train_df.head()

,0,1,2,3
0,95,female,colon,3
1,82,male,colon,2
2,27,female,colon,1
3,12,female,breast,4
4,68,female,colon,4


In [12]:
test_df = pd.read_csv(test_location, header=None)
test_df.head()

,0,1,2,3
0,25,male,lung,3
1,57,male,prostate,4
2,89,female,colon,3
3,35,male,prostate,1
4,6,female,colon,1
